In [1]:
import fiona 
from shapely.geometry import shape, Polygon, mapping
import math

### Removing unwanted vertices

This next segment takes a shapefile, goes through each of the contained polygons and removes intermediate vertices that do not represent a corner point.

In [3]:
shape_path = r'data/buildings_20k.shp'

def azimuth(point1, point2):
   '''Azimuth between 2 shapely points. each point is expected to be a list or tuple of (x, y)'''
   angle = math.atan2(point2[0] - point1[0], point2[1] - point1[1])
   return math.degrees(angle)if angle>0 else math.degrees(angle) + 180

cleaned_polygons = []

with fiona.open(shape_path, 'r') as src:
    for feature in src:
        footprint = shape(feature['geometry'])

        previous_angle = None
        polygon_sequence = []
        for previous, current in zip(footprint.exterior.coords, footprint.exterior.coords[1:]):

            angle = azimuth(previous, current)
            if previous_angle:
                if abs(previous_angle - angle) < 10:
                    continue

            polygon_sequence += [previous]
            previous_angle = angle

        cleaned_polygons.append(Polygon(polygon_sequence))

schema = {'geometry': 'Polygon', 'properties': {}}
shape_out = "buildings_20k_cleaned.shp"
with fiona.open(shape_out, 'w', crs='epsg:4326', driver='ESRI Shapefile', schema=schema) as output:
    for poly in cleaned_polygons:
        output.write({'geometry': mapping(poly)})  



### Equalize vertex count on all polygons

In [28]:
from shapely.geometry import Polygon, LineString
import geopandas as gpd
import numpy as np

from shapely.geometry import LineString

def interpolate_vertices(polygon, num_vertices):
    new_coords = []
    num_edges = len(polygon.exterior.coords) - 1
    vertices_per_edge, leftover_vertices = divmod(num_vertices, num_edges)

    # keep first node
    new_coords.append(polygon.exterior.coords[0])

    # iterate all edges
    for i in range(num_edges):
        start = polygon.exterior.coords[i]
        end = polygon.exterior.coords[i + 1]
        line = LineString([start, end])
        
        num_interpolated_vertices = vertices_per_edge
        # consume any leftover vertices if there are any
        if leftover_vertices > 0:
            num_interpolated_vertices += 1
            leftover_vertices -= 1

        # calculate interpolation distance between points
        total_length = line.length
        segment_length = total_length / (num_interpolated_vertices + 1)

        # interpolate points along line
        interpolated_points = []
        for i in range(1, num_interpolated_vertices + 1):
            distance_along_line = segment_length * i
            point = line.interpolate(distance_along_line)
            interpolated_points.append(point)

        new_coords.extend(interpolated_points)
        new_coords.append(end)

    return Polygon(new_coords)

# Load the shapefile
data = gpd.read_file('buildings_20k_cleaned.shp')

# Iterate over each polygon
for index, row in data.iterrows():
    polygon = row['geometry']
    num_vertices_to_add = 64 - len(polygon.exterior.coords)

    if num_vertices_to_add > 0:
        new_polygon = interpolate_vertices(polygon, num_vertices_to_add)
        data.at[index, 'geometry'] = new_polygon

        
# Save the modified shapefile
data.to_file('buildings_20k_interpolated.shp')